# Plugins

> Optional plugin integration for extensible settings systems

In [ ]:
#| default_exp plugins

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from typing import Dict, Any, Optional, Protocol, runtime_checkable

# Import from cjm-fasthtml-plugins
from cjm_fasthtml_plugins.core.metadata import PluginMetadata

## Plugin Integration

This module provides integration with the `cjm-fasthtml-plugins` library for extensible settings systems.

- Uses `PluginMetadata` from `cjm-fasthtml-plugins`
- Defines `PluginRegistryProtocol` for registry compatibility
- Applications should use `UnifiedPluginRegistry` from `cjm-fasthtml-plugins` instead of `SimplePluginRegistry`

In [ ]:
# Cell removed - using PluginMetadata from cjm-fasthtml-plugins

In [ ]:
# Cell removed - PluginMetadata now imported from cjm-fasthtml-plugins
# The imported version includes additional features:
# - Execution mode tracking (local, cloud, subprocess, etc.)
# - Resource management (child PIDs, containers, cloud instances)
# - Lifecycle awareness support

## Plugin Registry Protocol

In [ ]:
#| export
@runtime_checkable
class PluginRegistryProtocol(Protocol):
    """Protocol that plugin registries should implement.
    
    This allows the settings library to work with any plugin system
    that implements these methods.
    """
    
    def get_plugin(self, unique_id: str) -> Optional[PluginMetadata]:
        """Get plugin metadata by unique ID."""
        ...
    
    def get_plugins_by_category(self, category: str) -> list[PluginMetadata]:
        """Get all plugins in a category."""
        ...
    
    def get_categories_with_plugins(self) -> list[str]:
        """Get all categories that have registered plugins."""
        ...
    
    def load_plugin_config(self, unique_id: str) -> Dict[str, Any]:
        """Load saved configuration for a plugin."""
        ...
    
    def save_plugin_config(self, unique_id: str, config: Dict[str, Any]) -> bool:
        """Save configuration for a plugin."""
        ...

## Plugin Registry

**Recommended:** Use `UnifiedPluginRegistry` from `cjm-fasthtml-plugins` for production applications.

The `SimplePluginRegistry` below is kept for backward compatibility and simple use cases. For applications that need:
- Multiple plugin categories (transcription, LLM, etc.)
- Plugin manager integration
- Advanced resource tracking

Use `UnifiedPluginRegistry` which implements this same protocol.

In [ ]:
#| export
class SimplePluginRegistry:
    """Simple implementation of PluginRegistryProtocol.
    
    This provides a basic plugin registry that can be used with the settings
    library. Applications with more complex needs can implement their own
    registry that follows the PluginRegistryProtocol.
    
    Categories are arbitrary strings defined by the application.
    """
    
    def __init__(self, config_dir: Optional[Path] = None):
        self._plugins: Dict[str, PluginMetadata] = {}
        self._config_dir = config_dir or Path("configs")
    
    def register_plugin(self, metadata: PluginMetadata):
        """Register a plugin."""
        # Check if plugin is configured
        config_file = self._config_dir / f"{metadata.get_unique_id()}.json"
        metadata.is_configured = config_file.exists()
        
        self._plugins[metadata.get_unique_id()] = metadata
    
    def get_plugin(self, unique_id: str) -> Optional[PluginMetadata]:
        """Get plugin metadata by unique ID."""
        return self._plugins.get(unique_id)
    
    def get_plugins_by_category(self, category: str) -> list:
        """Get all plugins in a category."""
        return [p for p in self._plugins.values() if p.category == category]
    
    def get_categories_with_plugins(self) -> list:
        """Get all categories that have registered plugins."""
        categories = set(p.category for p in self._plugins.values())
        return sorted(categories)
    
    def load_plugin_config(self, unique_id: str) -> Dict[str, Any]:
        """Load saved configuration for a plugin."""
        import json
        config_file = self._config_dir / f"{unique_id}.json"
        if config_file.exists():
            with open(config_file, 'r') as f:
                return json.load(f)
        return {}
    
    def save_plugin_config(self, unique_id: str, config: Dict[str, Any]) -> bool:
        """Save configuration for a plugin."""
        import json
        try:
            self._config_dir.mkdir(exist_ok=True, parents=True)
            config_file = self._config_dir / f"{unique_id}.json"
            with open(config_file, 'w') as f:
                json.dump(config, f, indent=2)
            
            # Update is_configured status
            if unique_id in self._plugins:
                self._plugins[unique_id].is_configured = True
            
            return True
        except Exception as e:
            print(f"Error saving plugin config: {e}")
            return False

In [ ]:
# Example: Using SimplePluginRegistry with custom categories
import tempfile

with tempfile.TemporaryDirectory() as tmpdir:
    registry = SimplePluginRegistry(config_dir=Path(tmpdir))
    
    # Register plugins with application-specific categories
    registry.register_plugin(PluginMetadata(
        name="csv_exporter",
        category="export",  # Custom category
        title="CSV Exporter",
        config_schema={
            "type": "object",
            "properties": {
                "delimiter": {"type": "string", "default": ","}
            }
        },
        description="Export data to CSV format"
    ))
    
    registry.register_plugin(PluginMetadata(
        name="json_exporter",
        category="export",  # Same custom category
        title="JSON Exporter",
        config_schema={
            "type": "object",
            "properties": {
                "indent": {"type": "integer", "default": 2}
            }
        }
    ))
    
    registry.register_plugin(PluginMetadata(
        name="data_cleaner",
        category="preprocessing",  # Different custom category
        title="Data Cleaner",
        config_schema={
            "type": "object",
            "properties": {
                "remove_nulls": {"type": "boolean", "default": True}
            }
        }
    ))
    
    print(f"Registered plugins: {list(registry._plugins.keys())}")
    print(f"Categories: {registry.get_categories_with_plugins()}")
    print(f"Export plugins: {[p.name for p in registry.get_plugins_by_category('export')]}")
    print(f"Preprocessing plugins: {[p.name for p in registry.get_plugins_by_category('preprocessing')]}")
    
    # Save and load config using correct unique_id format (category_name)
    unique_id = "export_csv_exporter"  # Format: category_name
    registry.save_plugin_config(unique_id, {"delimiter": ";"})
    loaded = registry.load_plugin_config(unique_id)
    print(f"\nSaved and loaded config: {loaded}")
    print(f"Is configured: {registry.get_plugin(unique_id).is_configured}")

Registered plugins: ['export_csv_exporter', 'export_json_exporter', 'preprocessing_data_cleaner']
Categories: ['export', 'preprocessing']
Export plugins: ['csv_exporter', 'json_exporter']
Preprocessing plugins: ['data_cleaner']

Saved and loaded config: {'delimiter': ';'}
Is configured: True


In [ ]:
# Example: Using UnifiedPluginRegistry from cjm-fasthtml-plugins
from cjm_fasthtml_plugins.core.registry import UnifiedPluginRegistry

# Mock plugin manager for demonstration
from dataclasses import dataclass as dc

@dc
class MockPluginData:
    name: str
    version: str

class MockPluginManager:
    def discover_plugins(self):
        return [
            MockPluginData("plugin_a", "1.0.0"),
            MockPluginData("plugin_b", "1.0.0")
        ]
    def get_plugin_config_schema(self, name: str):
        return {
            "type": "object",
            "title": f"{name.title()} Configuration",
            "properties": {
                "enabled": {"type": "boolean", "default": True}
            }
        }

with tempfile.TemporaryDirectory() as tmpdir:
    # Create unified registry
    registry = UnifiedPluginRegistry(config_dir=Path(tmpdir))
    
    # Register a plugin manager for a category
    registry.register_plugin_manager(
        category="data_processing",
        manager=MockPluginManager(),
        display_name="Data Processing"
    )
    
    # The registry implements PluginRegistryProtocol
    print(f"Implements protocol: {isinstance(registry, PluginRegistryProtocol)}")
    print(f"Categories: {registry.get_categories_with_plugins()}")
    print(f"Plugins in category: {[p.name for p in registry.get_plugins_by_category('data_processing')]}")
    
    # Get a specific plugin
    plugin = registry.get_plugin("data_processing_plugin_a")
    print(f"\nPlugin metadata:")
    print(f"  Name: {plugin.name}")
    print(f"  Category: {plugin.category}")
    print(f"  Title: {plugin.title}")
    print(f"  Unique ID: {plugin.get_unique_id()}")
    
    # Save and load config
    registry.save_plugin_config("data_processing_plugin_a", {"enabled": False})
    loaded = registry.load_plugin_config("data_processing_plugin_a")
    print(f"\nSaved and loaded config: {loaded}")

Implements protocol: True
Categories: ['data_processing']
Plugins in category: ['plugin_a', 'plugin_b']

Plugin metadata:
  Name: plugin_a
  Category: data_processing
  Title: Plugin_A Configuration
  Unique ID: data_processing_plugin_a

Saved and loaded config: {'enabled': False}


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()